In [6]:
observation_bones_list = [(0,11),
                    (0,12),
                    
                    (11,13),
                    (13,15),
                    (15,17),
                    (15,19),
                    (15,21),
                    (17,19),
                    
                    (12,14),
                    (14,16),
                    (16,18),
                    (16,20),
                    (16,22),
                    (18,20),
                    
                    (11,12),
                    (11,23),
                    (12,24),
                    (23,24),
                    
                    (23,25),
                    (25,27),
                    (27,29),
                    (27,31),
                    (29,31),
                    
                    (24,26),
                    (26,28),
                    (28,30),
                    (28,32),
                    (30,32)]

## basic

In [7]:
def angle_between_two_vectors(a, b, c, d):
    """
    점 A와 점 B를 이은 벡터와 점 C와 점 D를 이은 벡터가 이루는 각도를 계산합니다.
    
    Parameters:
    a, b, c, d (tuple or list): 네 점의 좌표 (x, y, z)
    
    Returns:
    float: 두 벡터가 이루는 각도 (라디안 단위)
    float: 두 벡터가 이루는 각도 (도 단위)
    """
    # 벡터 정의
    vec1 = np.array(b) - np.array(a)
    vec2 = np.array(d) - np.array(c)
    
    # 벡터의 내적
    dot_product = np.dot(vec1, vec2)
    
    # 벡터의 크기
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    # 코사인 값 계산
    cos_theta = dot_product / (norm1 * norm2)
    
    # 각도 계산 (라디안)
    angle_rad = np.arccos(cos_theta)
    
    # 각도 계산 (도)
    angle_deg = np.degrees(angle_rad)
    
    return angle_rad, angle_deg


In [22]:
import csv
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

def read_keypoints_from_csv(csv_file):
    keypoints = []
    world_keypoints = []
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            frame_keypoints = []
            frame_world_keypoints = []
            for i in range(33):
                x = float(row[1 + i*4])
                y = float(row[2 + i*4])
                z = float(row[3 + i*4])
                visibility = float(row[4 + i*4])
                frame_keypoints.append(landmark_pb2.NormalizedLandmark(x=x, y=y, z=z, visibility=visibility))
                
                world_x = float(row[133 + i*4])
                world_y = float(row[134 + i*4])
                world_z = float(row[135 + i*4])
                world_visibility = float(row[136 + i*4])
                frame_world_keypoints.append(landmark_pb2.NormalizedLandmark(x=world_x, y=world_y, z=world_z, visibility=world_visibility))
                
            keypoints.append(frame_keypoints)
            world_keypoints.append(frame_world_keypoints)
    return keypoints, world_keypoints

def extract_frame(video_path, frame_index):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        raise ValueError(f"Could not read frame {frame_index} from {video_path}")
    return frame

def draw_landmarks(image, landmarks, color):
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    pose_landmarks = landmark_pb2.NormalizedLandmarkList(landmark=landmarks)
    mp_drawing.draw_landmarks(image, pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))
    return image

import numpy as np

def angle_between_two_vectors(a, b, c, d):
    """
    점 A와 점 B를 이은 벡터와 점 C와 점 D를 이은 벡터가 이루는 각도를 계산합니다.
    
    Parameters:
    a, b, c, d (tuple or list): 네 점의 좌표 (x, y, z)
    
    Returns:
    float: 두 벡터가 이루는 각도 (라디안 단위)
    float: 두 벡터가 이루는 각도 (도 단위)
    """
    # 벡터 정의
    vec1 = np.array(b) - np.array(a)
    vec2 = np.array(d) - np.array(c)
    
    # 벡터의 내적
    dot_product = np.dot(vec1, vec2)
    
    # 벡터의 크기
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    # 코사인 값 계산
    cos_theta = dot_product / (norm1 * norm2)
    
    # 각도 계산 (라디안)
    angle_rad = np.arccos(cos_theta)
    
    # 각도 계산 (도)
    angle_deg = np.degrees(angle_rad)
    
    return angle_rad, angle_deg




def compare_and_visualize(video_file1, video_file2, csv_file1, csv_file2, frame1_index, frame2_index, output_image, distance_threshold=0.04):
    keypoints1, world_keypoints1 = read_keypoints_from_csv(csv_file1)
    keypoints2, world_keypoints2 = read_keypoints_from_csv(csv_file2)


    frame1_7 = extract_frame(video_file1, frame1_index)
    frame2_9 = extract_frame(video_file2, frame2_index)

    highlight_indices = []
    for bone in observation_bones_list:
        
        start_idx = bone[0]
        finish_idx = bone[1]
        
        
        A1 = np.array([world_keypoints1[frame1_index][start_idx].x, world_keypoints1[frame1_index][start_idx].y, world_keypoints1[frame1_index][start_idx].z])
        C1 = np.array([world_keypoints1[frame1_index][finish_idx].x, world_keypoints1[frame1_index][finish_idx].y, world_keypoints1[frame1_index][finish_idx].z])

        A2 = np.array([world_keypoints2[frame2_index][start_idx].x, world_keypoints2[frame2_index][start_idx].y, world_keypoints2[frame2_index][start_idx].z])
        C2 = np.array([world_keypoints2[frame2_index][finish_idx].x, world_keypoints2[frame2_index][finish_idx].y, world_keypoints2[frame2_index][finish_idx].z])

        _, angle = angle_between_two_vectors(A1, C1, A2, C2)
        print(angle)

        if angle> distance_threshold:
            highlight_indices.append(bone)
        
        print('angle_distance',highlight_indices)

    image1 = draw_landmarks(frame1_7, keypoints1[frame1_index], (0, 255, 255))
    image2 = draw_landmarks(frame2_9, keypoints2[frame2_index], (0, 255, 0))
    
    for idx in highlight_indices:
        for idx_pair in highlight_indices:
            start_landmark = keypoints2[frame2_index][idx_pair[0]]
            end_landmark = keypoints2[frame2_index][idx_pair[1]]
            start_point = (int(start_landmark.x * image2.shape[1]), int(start_landmark.y * image2.shape[0]))
            end_point = (int(end_landmark.x * image2.shape[1]), int(end_landmark.y * image2.shape[0]))
            cv2.line(image2, start_point, end_point, (0, 0, 255), 5)

    combined_image = np.concatenate((image1, image2), axis=1)

    #cv2.putText(combined_image, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, f'threshold: {round(distance_threshold,3)}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, 'Vector', (1700, 200), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 10, cv2.LINE_AA)
    cv2.imwrite(output_image, combined_image)

# 사용 예시
video_file1 = 'C:/Users/jk/action_assess_2/data/video/Z76/313-2-1-15-Z76_D.avi'
video_file2 = 'C:/Users/jk/action_assess_2/data/video/Z106/313-2-1-15-Z106_D.avi'
csv_file1 = 'C:/Users/jk/action_assess_2/data/csv/Z76/313-2-1-15-Z76_D.csv'
csv_file2 = 'C:/Users/jk/action_assess_2/data/csv/Z106/313-2-1-15-Z106_D.csv'
output_image = 'C:/Users/jk/action_assess_2/post_data/vector/output_image.png'

for threshold in np.arange(0, 45, 5):
    print('#'*100)
    output_image = f'C:/Users/jk/action_assess_2/post_data/vector/output_image_{round(threshold)}.png'
    compare_and_visualize(video_file1, video_file2, csv_file1, csv_file2, 4, 4, output_image, distance_threshold=threshold)

####################################################################################################
18.994264509940184
angle_distance [(0, 11)]
18.10154675895644
angle_distance [(0, 11), (0, 12)]
16.04513374937613
angle_distance [(0, 11), (0, 12), (11, 13)]
14.221676434367946
angle_distance [(0, 11), (0, 12), (11, 13), (13, 15)]
15.366523512400065
angle_distance [(0, 11), (0, 12), (11, 13), (13, 15), (15, 17)]
18.48208175779591
angle_distance [(0, 11), (0, 12), (11, 13), (13, 15), (15, 17), (15, 19)]
27.440028461649533
angle_distance [(0, 11), (0, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21)]
23.70854714540663
angle_distance [(0, 11), (0, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (17, 19)]
9.137187971196639
angle_distance [(0, 11), (0, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (17, 19), (12, 14)]
25.887597433966636
angle_distance [(0, 11), (0, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (17, 19), (12, 14), (14, 16)]
12.671020700886189
angl

In [ ]:
import numpy as np

def angle_between_two_vectors(a, b, c, d):
    """
    점 A와 점 B를 이은 벡터와 점 C와 점 D를 이은 벡터가 이루는 각도를 계산합니다.
    
    Parameters:
    a, b, c, d (tuple or list): 네 점의 좌표 (x, y, z)
    
    Returns:
    float: 두 벡터가 이루는 각도 (라디안 단위)
    float: 두 벡터가 이루는 각도 (도 단위)
    """
    # 벡터 정의
    vec1 = np.array(b) - np.array(a)
    vec2 = np.array(d) - np.array(c)
    
    # 벡터의 내적
    dot_product = np.dot(vec1, vec2)
    
    # 벡터의 크기
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    # 코사인 값 계산
    cos_theta = dot_product / (norm1 * norm2)
    
    # 각도 계산 (라디안)
    angle_rad = np.arccos(cos_theta)
    
    # 각도 계산 (도)
    angle_deg = np.degrees(angle_rad)
    
    return angle_rad, angle_deg

